In [3]:
import os
import math
import tiktoken
import re
import pandas as pd
import numpy as np
import torch
import torch.nn as nn

ModuleNotFoundError: No module named 'tiktoken'

In [2]:
pip install tiktoken

error: externally-managed-environment

× This environment is externally managed
╰─> To install Python packages system-wide, try brew install
    xyz, where xyz is the package you are trying to
    install.
    
    If you wish to install a Python library that isn't in Homebrew,
    use a virtual environment:
    
    python3 -m venv path/to/venv
    source path/to/venv/bin/activate
    python3 -m pip install xyz
    
    If you wish to install a Python application that isn't in Homebrew,
    it may be easiest to use 'pipx install xyz', which will manage a
    virtual environment for you. You can install pipx with
    
    brew install pipx
    
    You may restore the old behavior of pip by passing
    the '--break-system-packages' flag to pip, or by adding
    'break-system-packages = true' to your pip.conf file. The latter
    will permanently disable this error.
    
    If you disable this error, we STRONGLY recommend that you additionally
    pass the '--user' flag to pip, or set 

In [25]:
source_file = '/Users/arjunkrishna/Desktop/tfr_llm/the-hundred-page-machine-learning--.txt'

with open(source_file, 'r', encoding='utf-8') as f:
    raw_data = f.read()

preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_data)
preprocessed = [item.strip() for item in preprocessed if item.strip()]

# split into 90/10
n = len(preprocessed)
train_data = preprocessed[:int(n*0.9)]
val_data = preprocessed[int(n*0.9):]

train_text = " ".join(train_data) #'text': 'list' object cannot be converted to 'PyString' 
val_text = " ".join(val_data) # so we romove the list by using join

# tokenize using gpt2 
enc = tiktoken.get_encoding("gpt2")
train_ids = enc.encode_ordinary(train_text)
val_ids = enc.encode_ordinary(val_text)

torch.save(torch.tensor(train_ids), 'train.pt')
torch.save(torch.tensor(val_ids), 'train.pt')

print(f"train has {len(train_ids):,} tokens")
print(f"val has {len(val_ids):,} tokens")



train has 62,186 tokens
val has 6,902 tokens


In [ ]:
import torch 
from torch.utils.data import dataset, dataloader
class DatasetV1(Dataset):
    def __init__(self, txt, max_length, stride):
        self.input_ids = []
        self.target_ids = []
        token_ids = enc.encode(txt)
        for i in range(0, len(token_ids) - max_length, stride):
        

In [27]:
# attention mechanism

In [30]:
import torch
inputs = torch.tensor(
[[0.43, 0.15, 0.89], # your     (x^1)
 [0.55, 0.87, 0.66], # journey  (x^2)
 [0.55, 0.87, 0.66], # starts   (x^3)
 [0.22, 0.58, 0.33], # with     (x^4)
 [0.77, 0.25, 0.10], # one      (x^5)
 [0.05, 0.80, 0.55]] # step     (x^6)
)

In [37]:
query = inputs[1]
attn_scores_2 = torch.empty(inputs.shape[0])
for i, x_i in enumerate(inputs):
    print(i , x_i)
    attn_scores_2[i] = torch.dot(x_i, query)
print(attn_scores_2)

0 tensor([0.4300, 0.1500, 0.8900])
1 tensor([0.5500, 0.8700, 0.6600])
2 tensor([0.5500, 0.8700, 0.6600])
3 tensor([0.2200, 0.5800, 0.3300])
4 tensor([0.7700, 0.2500, 0.1000])
5 tensor([0.0500, 0.8000, 0.5500])
tensor([0.9544, 1.4950, 1.4950, 0.8434, 0.7070, 1.0865])


In [38]:
attn_weights_2_tmp = attn_scores_2 / attn_scores_2.sum() # normalizing the attention scores 
print("Attention weights:", attn_weights_2_tmp)
print("Sum:", attn_weights_2_tmp.sum())

Attention weights: tensor([0.1450, 0.2272, 0.2272, 0.1282, 0.1074, 0.1651])
Sum: tensor(1.0000)


In [40]:
def softmax_naive(x):
    return torch.exp(x) / torch.exp(x).sum(dim=0)
attn_weights_2_naive = softmax_naive(attn_scores_2)
print("Attention weights:", attn_weights_2_naive)
print("Sum:", attn_weights_2_naive.sum())

Attention weights: tensor([0.1379, 0.2368, 0.2368, 0.1234, 0.1077, 0.1574])
Sum: tensor(1.0000)


In [41]:
attn_weights_2 = torch.softmax(attn_scores_2, dim=0)
print("Attention weights:", attn_weights_2)
print("Sum:", attn_weights_2.sum())

Attention weights: tensor([0.1379, 0.2368, 0.2368, 0.1234, 0.1077, 0.1574])
Sum: tensor(1.0000)


In [42]:
# calculating context vectore with computed normalized attention weights
query = inputs[1]
context_vec_2 = torch.zeros(query.shape)
for i,x_i in enumerate(inputs):
    context_vec_2 += attn_weights_2[i]*x_i
print(context_vec_2)

tensor([0.4377, 0.6571, 0.5734])


In [44]:
attn_scores = torch.empty(6, 6) # attention scores
for i, x_i in enumerate(inputs):
    for j, x_j in enumerate(inputs):
        attn_scores[i, j] = torch.dot(x_i, x_j)
print(attn_scores)

tensor([[0.9995, 0.9544, 0.9544, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4950, 0.8434, 0.7070, 1.0865],
        [0.9544, 1.4950, 1.4950, 0.8434, 0.7070, 1.0865],
        [0.4753, 0.8434, 0.8434, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7070, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0865, 0.6565, 0.2935, 0.9450]])


In [46]:
attn_scores = inputs @ inputs.T # for loops are generally slow so we use matrix multplication
print(attn_scores)

tensor([[0.9995, 0.9544, 0.9544, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4950, 0.8434, 0.7070, 1.0865],
        [0.9544, 1.4950, 1.4950, 0.8434, 0.7070, 1.0865],
        [0.4753, 0.8434, 0.8434, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7070, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0865, 0.6565, 0.2935, 0.9450]])


In [47]:
attn_weights = torch.softmax(attn_scores, dim=-1) # normalize  so that the values in each row sum to 1
print(attn_weights)

tensor([[0.2093, 0.2001, 0.2001, 0.1239, 0.1218, 0.1448],
        [0.1379, 0.2368, 0.2368, 0.1234, 0.1077, 0.1574],
        [0.1379, 0.2368, 0.2368, 0.1234, 0.1077, 0.1574],
        [0.1431, 0.2068, 0.2068, 0.1458, 0.1259, 0.1716],
        [0.1529, 0.1962, 0.1962, 0.1369, 0.1882, 0.1297],
        [0.1377, 0.2171, 0.2171, 0.1413, 0.0983, 0.1885]])


In [48]:
row_2_sum = sum([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
print("Row 2 sum:", row_2_sum)
print("All row sums:", attn_weights.sum(dim=-1))

Row 2 sum: 1.0
All row sums: tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000])


In [54]:
all_context_vecs = attn_weights @ inputs # use attn_weights to compute all context vectors via mtx multplication
print(all_context_vecs)

tensor([[0.4384, 0.5977, 0.5831],
        [0.4377, 0.6571, 0.5734],
        [0.4377, 0.6571, 0.5734],
        [0.4266, 0.6346, 0.5554],
        [0.4630, 0.5945, 0.5303],
        [0.4142, 0.6558, 0.5693]])


In [55]:
print("Previous 2nd context vector:", context_vec_2)

Previous 2nd context vector: tensor([0.4377, 0.6571, 0.5734])
